In [3]:
# import sys
# del sys.modules['os']
import warnings
warnings.filterwarnings('ignore')
import os 
os.chdir("C:\\Users\\chiehhsin\\Desktop\\_Uchicago\\_Big Data\\final project\\pubg")

In [4]:
os.getcwd()

'C:\\Users\\chiehhsin\\Desktop\\_Uchicago\\_Big Data\\final project\\pubg'

EDA：<br>
Match based:(agg) <br>
Kai
- Game time: more players and less waiting time / by match type
- Average match time / by match type / party size
- How many matches in the dataset  / by match type / party size
- Average distance walk/ride 

Sheena
- team average survive time / team ranking
- Survival time distribution
- Death position for each match by time
- 落地成盒 (death <4 min)

`User based:` <br>
JC
- How many games, how many kills(agg)
- Average survival time(join two tables)
- Favorite weapon / map
- Movement tracking ???


In [1]:
# !pip install findspark
# import findspark
# findspark.init()

In [5]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession 
from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql import HiveContext

In [7]:
spark = SparkSession.builder.enableHiveSupport().appName('ReadWriteData').getOrCreate()
sc = spark.sparkContext

In [8]:
agg0 = spark.read.csv('agg/agg_match_stats_0.csv', inferSchema=True, header=True)
agg1 = spark.read.csv('agg/agg_match_stats_1.csv', inferSchema=True, header=True)
agg2 = spark.read.csv('agg/agg_match_stats_2.csv', inferSchema=True, header=True)
agg3 = spark.read.csv('agg/agg_match_stats_3.csv', inferSchema=True, header=True)
agg4 = spark.read.csv('agg/agg_match_stats_4.csv', inferSchema=True, header=True)
agg0.printSchema()

root
 |-- date: string (nullable = true)
 |-- game_size: integer (nullable = true)
 |-- match_id: string (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_placement: integer (nullable = true)



In [10]:
agg0.count()

13849287

In [10]:
aggr = agg0.union(agg1)
aggr = aggr.union(agg2)
aggr = aggr.union(agg3)
aggr = aggr.union(agg4)

In [11]:
aggr.count()

67369231

In [12]:
aggr.printSchema()

root
 |-- date: string (nullable = true)
 |-- game_size: integer (nullable = true)
 |-- match_id: string (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_placement: integer (nullable = true)



In [23]:
kil0 = spark.read.csv('deaths/kill_match_stats_final_0.csv', inferSchema=True, header=True)
kil1 = spark.read.csv('deaths/kill_match_stats_final_1.csv', inferSchema=True, header=True)
kil2 = spark.read.csv('deaths/kill_match_stats_final_2.csv', inferSchema=True, header=True)
kil3 = spark.read.csv('deaths/kill_match_stats_final_3.csv', inferSchema=True, header=True)
kil4 = spark.read.csv('deaths/kill_match_stats_final_4.csv', inferSchema=True, header=True)
kil0.printSchema()

root
 |-- killed_by: string (nullable = true)
 |-- killer_name: string (nullable = true)
 |-- killer_placement: double (nullable = true)
 |-- killer_position_x: double (nullable = true)
 |-- killer_position_y: double (nullable = true)
 |-- map: string (nullable = true)
 |-- match_id: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- victim_name: string (nullable = true)
 |-- victim_placement: double (nullable = true)
 |-- victim_position_x: double (nullable = true)
 |-- victim_position_y: double (nullable = true)



In [57]:
kil0.show(5)

+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|   killed_by|     killer_name|killer_placement|killer_position_x|killer_position_y|    map|            match_id|time|    victim_name|victim_placement|victim_position_x|victim_position_y|
+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|     Grenade| KrazyPortuguese|             5.0|         657725.1|         146275.2|MIRAMAR|2U4GBNA0YmnLSqvEy...| 823|KrazyPortuguese|             5.0|         657725.1|         146275.2|
|      SCAR-L|nide2Bxiaojiejie|            31.0|         93091.37|         722236.4|MIRAMAR|2U4GBNA0YmnLSqvEy...| 194|    X3evolution|            33.0|         92238.68|         723375.1|
|        S686|        Ascholes|            43.0|         366

In [22]:
kill = kil0.union(kil1)
kill = kill.union(kil2)
kill = kill.union(kil3)
kill = kill.union(kil4)

NameError: name 'kil0' is not defined

In [13]:
match_id = aggr.select('match_id').distinct()

In [21]:
match_id_kill = kill.select('match_id').distinct()
match_id = match_id.union(match_id_kill)

NameError: name 'kill' is not defined

In [15]:
match_id  = match_id.withColumn("monotonically_increasing_id", F.monotonically_increasing_id()) 

In [16]:
match_id.show(10)

+--------------------+---------------------------+
|            match_id|monotonically_increasing_id|
+--------------------+---------------------------+
|2U4GBNA0Yml0XDizI...|                          0|
|2U4GBNA0Ymmi_1k_l...|                          1|
|2U4GBNA0Ymkk4WNxi...|                          2|
|2U4GBNA0YmkjdRrCs...|                          3|
|2U4GBNA0YmlZtWxn9...|                          4|
|2U4GBNA0YmkkAIEnc...|                          5|
|2U4GBNA0Yml8haOPf...|                          6|
|2U4GBNA0Ymnu7BK9x...|                          7|
|2U4GBNA0Ymnh6-KFA...|                          8|
|2U4GBNA0Ymk4ggpim...|                          9|
+--------------------+---------------------------+
only showing top 10 rows



In [19]:
aggr_new = aggr.join(match_id, on=['match_id'], how='left').drop('match_id')

In [20]:
aggr_new.printSchema()

root
 |-- date: string (nullable = true)
 |-- game_size: integer (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_placement: integer (nullable = true)
 |-- monotonically_increasing_id: long (nullable = true)

